In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries
from functools import partial

# Save Results

## Load mlflow runs

In [2]:
db_port = 5001
db_name = 'recursive_clustering'
w = 'clust9'
# url = f'postgresql://{w}.ceremade.dauphine.lan:{db_port}/{db_name}'
url = f'postgresql://belucci@localhost:{db_port}/{db_name}'
engine = create_engine(url)
query = 'SELECT experiments.name from experiments'
experiment_names = pd.read_sql(query, engine)['name'].tolist()
# results_dir = Path('~/tab_benchmark/results')
# os.makedirs(results_dir, exist_ok=True)

In [3]:
experiment_names

['Default',
 'blob_experiment',
 'hpo_classif_experiment',
 'hpo_openml_experiment',
 'hpo_gaussian_experiment',
 'hpo_openml_final',
 'outlier_hc',
 'time_hc',
 'hpo_n_clusters',
 'hpo_classif_huge',
 'test']

In [4]:
params_columns = [
    'model_nickname',
    # 'n_samples',
    # 'n_features',
    # 'n_classes',
    # 'seed_model',
    # 'dataset_id',
    # 'dataset_name',
    # 'model_params/representative_method',
    # 'model_params/n_samples_representative',
    # 'model_params/kernel_kmeans',
    # 'direction',
    # 'hpo_metric',
    # 'standardize'
]

In [5]:
# latest_metrics_columns = [
#     'fit_model_return_elapsed_time',
#     'max_memory_used',
#     'n_clusters_',
#     'rand_score',
#     'adjusted_rand',
#     'mutual_info',
#     'adjusted_mutual_info',
#     'normalized_mutual_info',
#     'homogeneity',
#     'completeness',
#     'v_measure',
#     'silhouette',
#     'calinski_harabasz_score',
#     'davies_bouldin_score',
#     'inertia_score',
#     'best_n_clusters_',
#     'best_rand_score',
#     'best_adjusted_rand',
#     'best_mutual_info',
#     'best_adjusted_mutual_info',
#     'best_normalized_mutual_info',
#     'best_homogeneity_completeness_v_measure',
#     'best_silhouette',
#     'best_calinski_harabasz_score',
#     'best_davies_bouldin_score',
#     'best_inertia_score',
#     'best_homogeneity',
#     'best_completeness',
#     'best_v_measure',
# ]

In [6]:
# tags_columns = [
#     'raised_exception',
#     'EXCEPTION',
#     'parent_run_id',
#     'best_child_run_id',
# ]

In [7]:
# # parameters of each model, they are saved as tags for the parent run
# parameters = {
#     'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
#     'KMeans': ['n_clusters'],
#     'HDBSCAN': ['min_cluster_size'],
#     'DBSCAN': ['eps', 'min_samples'],
#     'AffinityPropagation': ['damping'],
#     'OPTICS': ['min_samples'],
#     'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
#     'WardAgglomerativeClustering': ['n_clusters'],
#     'Clique': ['n_partitions', 'density_threshold'],
#     'IRFLLRR': ['sc_n_clusters', 'p', 'c', 'lambda_', 'alpha'],
#     'KMeansProj': ['n_clusters'],
#     'Proclus': ['n_clusters', 'avg_dims'],
#     'SpectralClustering': ['n_clusters'],
#     'SingleAgglomerativeClustering': ['n_clusters'],
#     'CompleteAgglomerativeClustering': ['n_clusters'],
#     'AverageAgglomerativeClustering': ['n_clusters'],
# }
# all_model_parameters = [p for model, params in parameters.items() for p in params]
# all_model_parameters = list(set(all_model_parameters))
# for param in all_model_parameters:
#     tags_columns.append(param)

In [8]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiment_names, other_table=other_table, other_table_keys=other_table_keys)
# df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
# df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [9]:
df = df_params.copy()
df = df.loc[df['model_nickname']=='Clique']

In [10]:
df

key,status,start_time,end_time,model_nickname
run_uuid,,,,
0001d46d8dd948bbad09099d235a9da6,FINISHED,1734312486405,1.734394e+12,Clique
0004bd6bdf714a1cb2f9507300caced0,FINISHED,1734323304174,1.734575e+12,Clique
0009995930af4bf2ad237e91afd2c2e6,FINISHED,1734312595669,1.734395e+12,Clique
0009b3855a9a4e2c94689afc2522fcef,FINISHED,1734316897707,1.734506e+12,Clique
00114a96c32b458c8065249bf4e93d73,FINISHED,1734326163749,1.734609e+12,Clique
...,...,...,...,...
fffdb55f4c3e4ccb9427302a704f2b31,FINISHED,1734326893190,1.734654e+12,Clique
fffdd24819c14db39e166876c7ca68bb,FINISHED,1734322746496,1.734580e+12,Clique
fffe07c98e30453f972bc35ba0e5887a,FINISHED,1734306520814,1.734383e+12,Clique


In [11]:
run_ids = list(df.index)

In [12]:
runs_to_delete = run_ids

In [13]:
runs_to_delete

['0001d46d8dd948bbad09099d235a9da6',
 '0004bd6bdf714a1cb2f9507300caced0',
 '0009995930af4bf2ad237e91afd2c2e6',
 '0009b3855a9a4e2c94689afc2522fcef',
 '00114a96c32b458c8065249bf4e93d73',
 '00142bc910f84117a1c24c8cb363fdcd',
 '001624fa4d594f02bbbc3dfc30e70d61',
 '001941be718947d4b79efe4d9c834d38',
 '0019a195a7284e689dc2547f317d5771',
 '0021b167dcb84bd8a185eb166b95fc78',
 '0023a186f2b343c6a6a788e4a5121429',
 '00271980c3c44f9ab4d79ea2748fda78',
 '002c0f21b0034cea823013f8f7313d7a',
 '002c0f3cde2a4925a8ad149d11f1a6b3',
 '002c9aaa4c0c463fae9b197c2d125bcd',
 '002d097fd8cc4b5a93ac647a0e81c3f0',
 '002e14a0e3ff469a83ad6084f5de2211',
 '002f19d926e44245a523370afd9a0b5e',
 '002f98a0c079448fbbda46e665491220',
 '0030588963c247c09027481657bd2cca',
 '003281f9b64c43d1aef993aa6b5a6d2a',
 '00342010dc1a4db381b527b7cb7e8938',
 '003a0796b19a442e908a43cc1c8e7571',
 '003bd5e7f467485ca4381ddd72a7efd5',
 '003cab1df8ea48f1ac33c3a49b2d446d',
 '004224f198b0470f9c9054c4b33c4019',
 '0043c1eaa97a4c0bbeb79e2d83ef528b',
 

In [14]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [15]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

In [ ]:
query = f"""
DELETE
FROM
	experiment_tags
WHERE
	experiment_id = ANY(
	SELECT
		experiment_id
	FROM
		experiments
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	latest_metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	params
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	tags
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	runs
WHERE
	lifecycle_stage = 'deleted';

DELETE
FROM
	experiments
WHERE
	lifecycle_stage = 'deleted';
"""

with engine.begin() as conn:
    conn.execute(text(query))